# Training a NN to Improve Raw ORB/RANSAC Outputs

In [1]:
# Read in data from nn_X.csv and nn_Y.csv. Train a neural network on the data.

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Read in data. Ignore the chain_id column and the i column.
X = pd.read_csv('nn_X.csv').drop(columns=['chain_id', 'i']).to_numpy()
Y = pd.read_csv('nn_Y.csv').drop(columns=['chain_id', 'i']).to_numpy()


# DATA NORMALISATION
# Normalise the range column from [20, 600] to [0, 1].
X[:, 0] = (X[:, 0] - 20) / (600 - 20)

# Normalise the x, y, z columns from [-1000, 1000] to [-1, 1].
X[:, 1:4] = (X[:, 1:4] - (-1000)) / (1000 - (-1000))
Y[:, 0:3] = (Y[:, 0:3] - (-1000)) / (1000 - (-1000))

# FEATURE ENGINEERING
# For each of the columns x, y, z, qw, qx, qy, qz, add a column x^2, y^2, z^2, etc.
# This is to help it learn formulae that depend on the square of these values.
X = np.hstack([X, X[:, 3:]**2])

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# Convert data to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
Y_train = tf.convert_to_tensor(Y_train, dtype=tf.float32)
Y_test = tf.convert_to_tensor(Y_test, dtype=tf.float32)

# Create neural network
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(Y.shape[1])
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train neural network
model.fit(X_train, Y_train, epochs=200, batch_size=25, validation_data=(X_test, Y_test), verbose=2)

# Evaluate neural network
Y_pred = model.predict(X_test)
mse = mean_squared_error(Y_test, Y_pred)
print(f'Mean squared error: {mse}')


/var/folders/j2/ry46s6kx2l5d1gxm0w8kj5rc0000gn/T/ipykernel_19175/2017776461.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-04-19 19:10:15.368312: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/200
32/32 - 2s - loss: 0.0703 - val_loss: 0.0237 - 2s/epoch - 51ms/step
Epoch 2/200
32/32 - 0s - loss: 0.0333 - val_loss: 0.0199 - 145ms/epoch - 5ms/step
Epoch 3/200
32/32 - 0s - loss: 0.0300 - val_loss: 0.0188 - 148ms/epoch - 5ms/step
Epoch 4/200
32/32 - 0s - loss: 0.0278 - val_loss: 0.0196 - 141ms/epoch - 4ms/step
Epoch 5/200
32/32 - 0s - loss: 0.0271 - val_loss: 0.0206 - 147ms/epoch - 5ms/step
Epoch 6/200
32/32 - 0s - loss: 0.0255 - val_loss: 0.0188 - 151ms/epoch - 5ms/step
Epoch 7/200
32/32 - 0s - loss: 0.0244 - val_loss: 0.0185 - 139ms/epoch - 4ms/step
Epoch 8/200
32/32 - 0s - loss: 0.0236 - val_loss: 0.0203 - 140ms/epoch - 4ms/step
Epoch 9/200
32/32 - 0s - loss: 0.0231 - val_loss: 0.0188 - 145ms/epoch - 5ms/step
Epoch 10/200
32/32 - 0s - loss: 0.0237 - val_loss: 0.0192 - 142ms/epoch - 4ms/step
Epoch 11/200
32/32 - 0s - loss: 0.0225 - val_loss: 0.0191 - 137ms/epoch - 4ms/step
Epoch 12/200
32/32 - 0s - loss: 0.0225 - val_loss: 0.0187 - 135ms/epoch - 4ms/step
Epoch 13/200
32

# Saving the Model

In [13]:
# Save the model.
# I have to use the .h5 format is used instead of the newer .keras format because the latter is not supported by the legacy Keras/TensorFlow 2.15.0 API.
tf.saved_model.save(model, 'my_model.h5')

INFO:tensorflow:Assets written to: my_model.h5/assets


INFO:tensorflow:Assets written to: my_model.h5/assets
